In [1]:
# %load /Users/hwayment/ipynb_defaults.py
%load_ext autoreload
%autoreload 2

%pylab inline
import numpy as np
import pandas as pd

import seaborn as sns
sns.set_style('ticks')
sns.set_context('paper')

from arnie.mfe import mfe
from arnie.pfunc import pfunc
from arnie.bpps import bpps
from ipynb.draw import draw_struct
from RiboGraphViz import RGV
openai  ='open -a "Adobe Illustrator"'
import os, sys
import eternabench as eb

Populating the interactive namespace from numpy and matplotlib


In [2]:
zscores = pd.read_csv(os.environ['ETERNABENCH_PATH']+'/scoring_data/EternaBench_external_zscores_by_dataset.csv')
zscores = zscores.loc[zscores.window_size==900]

zscores['col_to_print'] = zscores.apply(lambda row: "%.2f (%.2f, %.2f)" % (row['pearson_mean'], 
                                                                           row['pearson_percentile_2.5'], row['pearson_percentile_97.5']), axis=1)

In [56]:
tmp = zscores[['package','Dataset','col_to_print']].pivot(columns=['package'],index='Dataset')
tmp.columns = [x[1] for x in tmp.columns]

tmp2 = pd.DataFrame()
for ds in zscores.Dataset.unique():
    tmp2 = tmp2.append(tmp.loc[ds])
    
tmp2 = tmp2[['eternafold_B','contrafold_2', 'rnasoft_blstar','vienna_2_60C','rnastructure_60C','vienna_2','rnastructure']]
tmp2.to_csv('MainText_Table2_precursor.csv')

In [5]:
ctr=0
for ds in zscores.Dataset.unique():
    tmp = zscores.loc[zscores.Dataset==ds]
    not_efold = tmp.loc[tmp.package!='eternafold_B']
    efold_min_cutoff = tmp.loc[tmp.package=='eternafold_B']['pearson_percentile_2.5'].values[0]
    if all(efold_min_cutoff > not_efold['pearson_percentile_97.5'].values):
        print('YES', ds)
        ctr+=1
    else:
        print('NO', ds)
        
        for pkg in ['contrafold_2', 'rnasoft_blstar','vienna_2_60C','rnastructure_60C','vienna_2','rnastructure']:
            pkg_max_cutoff = tmp.loc[tmp.package==pkg]['pearson_percentile_97.5'].values[0]
            if efold_min_cutoff < pkg_max_cutoff:
                print('\t%s tied, %.2f, %.2f' % (pkg, efold_min_cutoff, pkg_max_cutoff))
            not_pkg = tmp.loc[tmp.package!=pkg]
            pkg_min_cutoff = tmp.loc[tmp.package==pkg]['pearson_percentile_2.5'].values[0]   
            if all(pkg_min_cutoff > not_pkg['pearson_percentile_97.5'].values):
                print('BUT: %s won' % pkg)

print('FINAL COUNT', ctr)

YES Dadonaite,2019
YES Huber, 2019 in virion
NO Huber, 2019 in vitro
	contrafold_2 tied, 0.18, 0.20
	rnasoft_blstar tied, 0.18, 0.20
	vienna_2_60C tied, 0.18, 0.19
	rnastructure_60C tied, 0.18, 0.19
	vienna_2 tied, 0.18, 0.18
	rnastructure tied, 0.18, 0.18
YES Kutchko,2018
YES Lavender,2015
YES Siegfried,2014
YES Simon,2019 Invitro
NO Simon,2019 Invivo
	contrafold_2 tied, 0.48, 0.51
	rnasoft_blstar tied, 0.48, 0.49
	rnastructure_60C tied, 0.48, 0.49
NO Watters,2018
	contrafold_2 tied, -0.04, 0.04
	rnasoft_blstar tied, -0.04, 0.07
	vienna_2_60C tied, -0.04, 0.05
	rnastructure_60C tied, -0.04, 0.05
	vienna_2 tied, -0.04, 0.08
	rnastructure tied, -0.04, 0.08
YES Watts,2009
YES Huston, 2021 SARS-CoV-2 in vivo
NO Manfredonia, 2020 SARS-CoV-2 DMS in vitro
	rnasoft_blstar tied, 0.56, 0.56
YES Manfredonia, 2020 SARS-CoV-2 SHAPE in vitro
YES Manfredonia, 2020 SARS-CoV-2 SHAPE in vivo
YES Sun, 2021 SARS-CoV-2 in vitro
YES Sun, 2021 SARS-CoV-2 in vivo
NO Leppek, 2021, Nluc mRNA 1M7 in vitro
	cont

In [4]:
len(zscores.Dataset.unique())

36